In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA
from scipy.sparse.linalg import lsqr, lsmr

# Solve

In [2]:
def OMP(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]
            gamma = np.linalg.solve(D_I[:j+1] @ D_I[:j+1].T, D_I[:j+1] @ y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# QR

In [3]:
def OMP_2(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = 0
        R = 0
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]

            # gram = D_I[:j+1] @ D_I[:j+1].T
            # cov =  D_I[:j+1] @ y
            # gamma = LA.cho_solve((LA.cho_factor(gram, overwrite_a=True)), 
            #                      cov, overwrite_b=True)

            if j == 0:
                Q, R = LA.qr(D[k][:, np.newaxis],
                            overwrite_a = False,
                            mode = 'economic')
                if debug:
                    print(LA.norm(D[k]))
                    print(Q.T @ Q)
                    print(R)
            else:
                Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 

            if debug:            
                print(Q.T.shape)
                print(y.shape)
            gamma = LA.solve_triangular(R, Q.T @ y)
            
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Manual QR (Gram-Schmidt)

In [4]:
def OMP_3(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                D_k_norm = LA.norm(D[k])
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(LA.norm(D[k]))
                    print(Q[:, 0].T @ Q[:, 0])
                    print(R)
                    
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = LA.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm
                

            if debug:            
                print(Q[:, :j+1].T.shape)
                print(y.shape)
            gamma = LA.solve_triangular(R[:j+1, :j+1], Q[:, :j+1].T @ y, overwrite_b = False)
            
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Manual solve (only for first sparse coef for now)

In [5]:
def OMP_4(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0
        Q = np.empty((D.shape[1], T_0))
        R = np.empty((T_0, T_0))
        
        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]

            if j == 0:
                # Q, R = LA.qr(D[k][:, np.newaxis],
                #             overwrite_a = False,
                #             mode = 'economic')
                D_k_norm = LA.norm(D[k])
                R[0, 0] = D_k_norm
                Q[:, 0] = D[k] / D_k_norm
                
                if debug:
                    print(LA.norm(D[k]))
                    print(Q[:, :j+1].T @ Q[:, 0])
                    print(R)

                gamma = (Q[:, :j+1].T @ y) / D_k_norm
                
            else:
                # Q, R = LA.qr_insert(Q, R, D[k][:, np.newaxis], j, which='col') 
                dot = Q[:, :j].T @ D[k]
                R[0:j, j] = dot
                q_j = D[k] - Q[:, :j] @ dot

                q_j_norm = LA.norm(q_j)
                R[j, j] = q_j_norm
                Q[:, j] = q_j / q_j_norm

                if debug:            
                    print(Q[:, :j+1].T.shape)
                    print(y.shape)
                gamma = LA.solve_triangular(R[:j+1, :j+1], Q[:, :j+1].T @ y, overwrite_b = False)
                

            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

In [6]:
def OMP_verif(code):
    print(code)
    I = np.nonzero(code)
    print(I)
    print(code[I])

In [7]:
X, y = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [8]:
X.shape

(50, 20)

In [9]:
y.shape

(50, 2)

In [10]:
b_numpy = OMP(y.T, 2, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(2,)
(50, 2)
(50,)
1675390.9154018096
(1,)
(50, 1)
(50,)
1817892.1699581572
(2,)
(50, 2)
(50,)
1455231.954008666


In [11]:
OMP_verif(b_numpy)

[[  0.         138.8699232    0.           0.           0.
    0.           0.           0.           0.           0.
    0.          96.02020336   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         139.58589021   0.           0.           0.
    0.           0.           0.          73.73795705   0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 0, 1, 1]), array([ 1, 11,  1,  8]))
[138.8699232   96.02020336 139.58589021  73.73795705]


In [12]:
b_scipy = OMP_2(y.T, 2, X.T, debug=True)

7.633996503399396
[[1.]]
[[7.6339965]]
(1, 50)
(50,)
(1,)
(50, 1)
(50,)
2449425.071243986
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
6.338597587156968
[[1.]]
[[-6.33859759]]
(1, 50)
(50,)
(1,)
(50, 1)
(50,)
1817892.1699581572
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.9540086663


In [13]:
OMP_verif(b_scipy)

[[  0.         138.8699232    0.           0.           0.
    0.           0.           0.           0.           0.
    0.          96.02020336   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         139.58589021   0.           0.           0.
    0.           0.           0.          73.73795705   0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 0, 1, 1]), array([ 1, 11,  1,  8]))
[138.8699232   96.02020336 139.58589021  73.73795705]


In [14]:
b_scipy2 = OMP_3(y.T, 2, X.T, debug=True)

7.633996503399396
0.9999999999999999
[[  7.6339965   96.02020336]
 [139.58589021  73.73795705]]
(1, 50)
(50,)
(1,)
(50, 1)
(50,)
2449425.071243986
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
6.338597587156968
1.0
[[  6.33859759  96.02020336]
 [139.58589021  73.73795705]]
(1, 50)
(50,)
(1,)
(50, 1)
(50,)
1817892.1699581572
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.954008666


In [15]:
OMP_verif(b_scipy2)

[[  0.         138.8699232    0.           0.           0.
    0.           0.           0.           0.           0.
    0.          96.02020336   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         139.58589021   0.           0.           0.
    0.           0.           0.          73.73795705   0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 0, 1, 1]), array([ 1, 11,  1,  8]))
[138.8699232   96.02020336 139.58589021  73.73795705]


In [16]:
b_scipy3 = OMP_4(y.T, 2, X.T, debug=True)

7.633996503399396
[1.]
[[  7.6339965   96.02020336]
 [139.58589021  73.73795705]]
(1,)
(50, 1)
(50,)
2449425.071243986
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1675390.9154018096
6.338597587156968
[1.]
[[  6.33859759  96.02020336]
 [139.58589021  73.73795705]]
(1,)
(50, 1)
(50,)
1817892.1699581572
(2, 50)
(50,)
(2,)
(50, 2)
(50,)
1455231.954008666


In [17]:
OMP_verif(b_scipy3)

[[  0.         138.8699232    0.           0.           0.
    0.           0.           0.           0.           0.
    0.          96.02020336   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         139.58589021   0.           0.           0.
    0.           0.           0.          73.73795705   0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 0, 1, 1]), array([ 1, 11,  1,  8]))
[138.8699232   96.02020336 139.58589021  73.73795705]


In [18]:
# b_scipy4 = OMP_5(y.T, 1, X.T, debug=True)

In [19]:
# OMP_verif(b_scipy4)

In [20]:
np.allclose(b_scipy[np.nonzero(b_scipy)], b_numpy[np.nonzero(b_numpy)])

True

In [21]:
np.allclose(b_scipy2[np.nonzero(b_scipy2)], b_numpy[np.nonzero(b_numpy)])

True

In [22]:
np.allclose(b_scipy3[np.nonzero(b_scipy3)], b_numpy[np.nonzero(b_numpy)])

True

In [23]:
# np.allclose(b_scipy4[np.nonzero(b_scipy4)], b_numpy[np.nonzero(b_numpy)])

## IT WORKS !!!

In [24]:
X, y = make_regression(n_samples = 50, n_features = 300, n_targets = 20_000, noise=4, random_state=0)

In [25]:
%timeit OMP(y.T, 1, X.T)

862 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit OMP_2(y.T, 1, X.T)

2.61 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit OMP_3(y.T, 1, X.T)

1.62 s ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit OMP_4(y.T, 1, X.T)

839 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
# %timeit OMP_5(y.T, 1, X.T)

In [30]:
%load_ext line_profiler
from line_profiler import profile

In [31]:
np_OMP = profile(OMP)
%lprun -f np_OMP np_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.4577 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_20760\3367727764.py
Function: OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        370.0    370.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                               #  

In [32]:
sci_OMP = profile(OMP_2)
%lprun -f sci_OMP sci_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 4.41265 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_20760\4221389306.py
Function: OMP_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_2(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        395.0    395.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     199869.0     10.0      0.5      for i, y in enumerate(Y):
     8     20000      60472.0      3.0      0.1          I = []
     9     20000     408216.0     20.4      0.9          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      43161.0      2.2      0.1          r = y
    11     20000      49220.0      2.5      0.1

In [33]:
sci_OMP2 = profile(OMP_3)
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 2.74009 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_20760\2272688001.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        364.0    364.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     186604.0      9.3      0.7      for i, y in enumerate(Y):
     8     20000      59296.0      3.0      0.2          I = []
     9     20000     382407.0     19.1      1.4          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      42344.0      2.1      0.2          r = y
    11     20000      41824.0      2.1      0.2

In [34]:
sci_OMP3 = profile(OMP_4)
%lprun -f sci_OMP3 sci_OMP3(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.19242 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_20760\2816955562.py
Function: OMP_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_4(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        329.0    329.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     164882.0      8.2      1.4      for i, y in enumerate(Y):
     8     20000      57976.0      2.9      0.5          I = []
     9     20000     357191.0     17.9      3.0          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      43274.0      2.2      0.4          r = y
    11     20000      44989.0      2.2      0.4

In [35]:
# sci_OMP4 = profile(OMP_5)
# %lprun -f sci_OMP4 sci_OMP4(y.T, 1, X.T)